#Compétition 2
Anshita Saxena, Denis Lemarchand ()

#Code

In [ ]:
import numpy as np
import pandas as pd

##Upload Kaggle File from Google Drive

In [ ]:
#@title Install PyDrive
!pip install PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#This notebook should access to Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# train data
# https://drive.google.com/file/d/17cD6hFS_AfKxtfjKxfeNUkC44jfSYkGj/view?usp=sharing
# train result
# https://drive.google.com/file/d/1iEpwQ3B4d4gIUl3ynq_p74U1QKZTURem/view?usp=sharing
# test data
# https://drive.google.com/file/d/1QbhqakgLpBXWr5sMFSe-BQpFw_X2XZZR/view?usp=sharing

#download Kaggle Files from Google Drive to store it in Colab Session
downloaded = drive.CreateFile({'id':"17cD6hFS_AfKxtfjKxfeNUkC44jfSYkGj"})   
downloaded.GetContentFile('train.csv') 

downloaded = drive.CreateFile({'id':"1iEpwQ3B4d4gIUl3ynq_p74U1QKZTURem"})   
downloaded.GetContentFile('train_result.csv')    

downloaded = drive.CreateFile({'id':"1QbhqakgLpBXWr5sMFSe-BQpFw_X2XZZR"})   
downloaded.GetContentFile('test.csv')

In [ ]:
# Mount your Google Drive to Collaboratory
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#
df_train = pd.read_csv('train.csv')
train = df_train.to_numpy()
train_inputs = train[:]
print(train_inputs.shape)

#
df_train_result = pd.read_csv('train_result.csv')
train_results = df_train_result.to_numpy()
train_labels = train_results[:,1]
print(train_labels.shape)

#
df_test = pd.read_csv('test.csv')
test = df_test.to_numpy()
test_inputs = test[:]
print(test_inputs.shape)

(1040323, 2)
(1040323,)
(560175, 2)


In [ ]:
#test
df_test.to_csv('gdrive/MyDrive/test_data_gdrive.csv', index=False)

In [ ]:
train_labels[train_labels=='negative']=0
train_labels[train_labels=='neutral']=1
train_labels[train_labels=='positive']=2

In [ ]:
# Smoke Tests
list_classes = np.unique(train_labels)
print(list_classes)

n = np.random.randint(0, len(train_inputs))
print(train_inputs[n], train_labels[n])

n = np.random.randint(0, len(test_inputs))
print(test_inputs[n])

dist_train_labels = [np.sum(train_labels==0)/len(train_labels),
  np.sum(train_labels==1)/len(train_labels),
  np.sum(train_labels==2)/len(train_labels)]

print(dist_train_labels)

[0 1 2]
[628269 '@BJermain thanx buddy '] 2
[317387 'i still want a red balloon. ']
[0.49965539548774757, 8.074415349848076e-05, 0.5002638603587539]


##Generate submission file

In [ ]:
#make submission file from numpy data
#The value of class should be 0, 1 or 2 with 0 being negative, 1 being neutral and 2 being positive class.
def PrepareKaggleFile(test_inputs, test_predictions, file='tests_label.csv'):
    output_data_for_kaggle = np.zeros((len(test_inputs),2))
    for i in range(len(test_inputs)):
      output_data_for_kaggle[i,0] = i
      output_data_for_kaggle[i,1] = test_predictions[i]

    output_data_for_kaggle = output_data_for_kaggle.astype(int)

    print(output_data_for_kaggle)

    df = pd.DataFrame(data=output_data_for_kaggle,columns=['id','target'])
    df.to_csv(file,index=False)

##LSTM

###Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train_result["target"].iloc[:1]

0    positive
Name: target, dtype: object

In [ ]:
df_train_result[df_train_result["target"]=='negative']=0
df_train_result[df_train_result["target"]=='neutral']=2 # As per IID 
df_train_result[df_train_result["target"]=='positive']=2

In [ ]:
pd_concat_df = pd.concat((df_train, df_train_result), axis = 1)

In [ ]:
pd_concat_df.target[pd_concat_df["target"]== 1] # Sanity check

Series([], Name: target, dtype: object)

In [ ]:
print(df_train_result["target"].iloc[0:10])

0    2
1    0
2    2
3    2
4    0
5    0
6    2
7    0
8    0
9    0
Name: target, dtype: object


In [ ]:
print(df_train.shape)
print(df_train_result.shape)

(1040323, 2)
(1040323, 2)


In [ ]:
df_train.head()

,id,text
0,0,Anyway Im getting of for a while
1,1,"My red, Apache isn't feelin too well this morn..."
2,2,@danyelljoy you should be its great. friday w...
3,3,its 11:30pm and i dont wanna sleep; so i debat...
4,4,Why does twitter eat my DM's? Not happy


In [ ]:
df_train_result.head()

,id,target
0,2,2
1,0,0
2,2,2
3,2,2
4,0,0


In [ ]:
import nltk                             
from nltk.corpus import twitter_samples   
import matplotlib.pyplot as plt           
import random  
import plotly.graph_objects as go

In [ ]:
nltk.download('stopwords')

import re                                  
import string                             
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df_train_clean = df_train.copy()

In [ ]:
# Converting all the upper case to lower case to avoid the distinction between them
df_train_clean['text'] = df_train_clean.text.str.lower()
# Putting the regex for removing the https and www URLs
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))

In [ ]:
# Remove the video and links
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'{link}', '', x))
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r"\[video\]", '', x))

In [ ]:
# Remove html reference characters
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'&[a-z]+;', '', x))

In [ ]:
#df_train_clean.head(20)

In [ ]:
# Remove usernames
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'@[^\s]+', '', x))

In [ ]:
#df_train_clean.head(20)

In [ ]:
# Removing numbers
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r'\d+', '', x))

In [ ]:
#df_train_clean.head(20)

In [ ]:
# Removing hashmarks, non-letter characters
#df_train_clean.text = df_train_clean.text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

In [ ]:
#df_train_clean.head(20)

In [ ]:
# Remove punctuation and numbers
#def remove_punct(text):
#    text  = "".join([char for char in text if char not in string.punctuation])
#    text = re.sub('[0-9]+', '', text)
#    return text

#df_train_clean['text'] = df_train_clean['text'].apply(lambda x: remove_punct(x))
#df_train_clean.head(10)

In [ ]:
# Remove stop words
#stopword_list = stopwords.words('english')
#df_train_clean['text'] = df_train_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopword_list)]))
#df_train_clean.head(20)

In [ ]:
#df_train_clean_before_stemmer = df_train_clean.copy()

In [ ]:
#df_train_clean = df_train_clean_before_stemmer.copy()
#df_train_clean.head()

In [ ]:
# Snowball Stemmer is also known as the Porter2 stemming algorithm because it is a better version of the Porter Stemmer. It is more aggressive than Porter Stemmer.
#from nltk.stem.snowball import SnowballStemmer
#snowBallStemmer = SnowballStemmer("english")
#def stemming(text):
#    text = [snowBallStemmer.stem(word) for word in text.split()]
#    return ' '.join(text)

#df_train_clean['text'] = df_train_clean['text'].apply(lambda x: stemming(x))
#df_train_clean.head()

In [ ]:
#df_train_clean.head(20)

In [ ]:
np.random.seed(1234)

In [ ]:
print(df_train_clean['text'][0])
print(df_train_clean['text'].shape)

anyway im getting of for a while 
(1040323,)


In [ ]:
print(df_train_result['target'][0])
print(df_train_result['target'].shape)

2
(1040323,)


###Prepare dataset, split, tokeniser

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 10000
tokenizer = Tokenizer(num_words=max_features, split=' ')

df_test_clean = df_test.copy()
# Converting all the upper case to lower case to avoid the distinction between them
df_test_clean['text'] = df_test_clean.text.str.lower()

df_all_text = df_train_clean['text'].append(df_test_clean['text'])

In [ ]:
tokenizer.fit_on_texts(df_all_text.values)

In [ ]:
print(pd_concat_df.shape)

(1040323, 4)


In [ ]:
#df_train, df_val, y_train, y_val = train_test_split(df_train_clean['text'], df_train_result['target'], random_state=42, test_size=0.1)

In [ ]:
#df_train = df_train.to_frame(name='text')
#df_val = df_val.to_frame(name='text')

df_train = df_train_clean['text']
y_train = df_train_result['target']

In [ ]:
#X_train = tokenizer.texts_to_sequences(df_train['text'].values)
X_train = tokenizer.texts_to_sequences(df_train.values)
X_train = pad_sequences(X_train)
#X_val = tokenizer.texts_to_sequences(df_val['text'].values)
#X_val = pad_sequences(X_val)

In [ ]:
#[X_train.shape,X_val.shape] #output : [(936290, 34), (104033, 27)]
X_train.shape

(1040323, 116)

In [ ]:
"""
Here, we need to match the X_val (i.e., 27) features to the X_train features 
(i.e., 34) therefore, we will apply padding of zeros.
"""
#import numpy as np
#X_val = np.lib.pad(X_val, ((0,0),(X_train.shape[1] - X_val.shape[1],0)), 'constant', constant_values=(0))
#X_val.shape #output : (104033, 34)

'\nHere, we need to match the X_val (i.e., 27) features to the X_train features \n(i.e., 34) therefore, we will apply padding of zeros.\n'

In [ ]:
#[X_train.shape,X_val.shape]

In [ ]:
#[y_train.shape, y_val.shape]
y_train.shape

(1040323,)

In [ ]:
y_train = y_train.to_frame(name='target')
#y_val = y_val.to_frame(name='target')
print(type(y_train))
#print(type(y_val))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
Y_train = np.array(pd.get_dummies(y_train)) #inverse is df.idxmax(axis=1)
#Y_val = np.array(pd.get_dummies(y_val))
Y_train.shape

(1040323, 2)

In [ ]:
pd_concat_df.head()

,id,text,id,target
0,0,Anyway Im getting of for a while,2,2
1,1,"My red, Apache isn't feelin too well this morn...",0,0
2,2,@danyelljoy you should be its great. friday w...,2,2
3,3,its 11:30pm and i dont wanna sleep; so i debat...,2,2
4,4,Why does twitter eat my DM's? Not happy,0,0


###Create model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [ ]:
max_features = 10000
embed_dim = 256
model = Sequential()
"""
our LSTM network is creating an embedding layer. An embedding layer allows us to
 convert each word into a fixed-length vector and that is a better way to 
 represent those words along with reduced dimensions. 
 The max_features parameter is the size of the vocabulary=set on the vocabulary.
 the embed_dim parameter is the length of the vector.
 for each word and input_length is the maximum length of a sequence.
"""
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))

In [ ]:
"""
The main purpose of the spatial dropout layer is to avoid overfitting and that 
is done by probabilistically removing the inputs of this layer (or the output of 
the embedding layer in the network we’re building).
In the end, the nodes of the network are more robust to the future inputs and 
tend to not overfit.
"""
model.add(SpatialDropout1D(0.2))

In [ ]:
"""
The first parameter lstm_out that we’ve defined as 256 it’s the dimensionality 
of the output space, and we can choose an even larger number trying to improve 
our model, but that can lead to many problems like overfitting and a long 
training time.
The dropout parameter is applied to the inputs and/or outputs of our model 
(the linear transformations), while the recurrent dropout is applied to the 
recurrent state, or cell state, of the model. Recurrent dropout affects the 
“memory” of the network. The small the dataset is, the small the value of 
recurrent_dropout.
"""
lstm_out = 512
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.1))

In [ ]:
model.add(Dense(2,activation='sigmoid'))
"""
An implementation of the Adam algorithm, which is a robust extended version of 
the stochastic gradient descent.
"""
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 116, 256)          2560000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 116, 256)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 512)               1574912   
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 4,135,938
Trainable params: 4,135,938
Non-trainable params: 0
_________________________________________________________________
None


###Training, save model, prediction and output file

In [ ]:
"""
The batch size is the number of samples to run through the network before a 
weight update is performed (an epoch), we’ll keep it low as it requires less 
memory.
"""
batch_size = 64
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2, shuffle=True)

16256/16256 - 6803s - loss: 0.3376 - accuracy: 0.8513 - 6803s/epoch - 419ms/step


In [ ]:
filename = 'gdrive/MyDrive/neural_network_lstm.h5'
model.save(filename)

In [ ]:
from tensorflow import keras
filename = 'gdrive/MyDrive/neural_network_lstm.h5'
model = keras.models.load_model(filename)

In [ ]:
df_test_clean = df_test.copy()
# Converting all the upper case to lower case to avoid the distinction between them
#df_test_clean['text'] = df_test_clean.text.str.lower()
df_test_clean = df_test_clean.str.lower()

# Putting the regex for removing the https and www URLs
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
# Remove the video and links
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r'{link}', '', x))
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r"\[video\]", '', x))
# Remove html reference characters
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
# Remove usernames
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r'@[^\s]+', '', x))
# Removing numbers
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r'\d+', '', x))
# Removing hashmarks, non-letter characters
#df_test_clean.text = df_test_clean.text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
# Remove punctuation and numbers
#def remove_punct(text):
#    text  = "".join([char for char in text if char not in string.punctuation])
#    text = re.sub('[0-9]+', '', text)
#    return text

#df_test_clean['text'] = df_test_clean['text'].apply(lambda x: remove_punct(x))
# Remove stop words
#stopword_list = stopwords.words('english')
#df_test_clean['text'] = df_test_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopword_list)]))
#df_test_clean_before_stemmer = df_test_clean.copy()
#df_test_clean = df_test_clean_before_stemmer.copy()
# Snowball Stemmer is also known as the Porter2 stemming algorithm because it is a better version of the Porter Stemmer. It is more aggressive than Porter Stemmer.
#from nltk.stem.snowball import SnowballStemmer
#snowBallStemmer = SnowballStemmer("english")
#def stemming(text):
#    text = [snowBallStemmer.stem(word) for word in text.split()]
#    return ' '.join(text)

#df_test_clean['text'] = df_test_clean['text'].apply(lambda x: stemming(x))
df_test_clean.head()

0    so cut... missed out on all the tce extreme me...
1       thank you very much wasin  this is awesome lol
2    squeezing in some internet time before dustin ...
3    @fanoou; oh i do have moral but i'm gonna be b...
4    @nsuissa   i just bought sammy some joint/hip ...
Name: text, dtype: object

In [ ]:
df_test = df_test_clean#df_test_clean['text']
X_test = tokenizer.texts_to_sequences(df_test.values)
X_test = pad_sequences(X_test)
X_test.shape

(560175, 40)

In [ ]:
import numpy as np
X_test = np.lib.pad(X_test, ((0,0),(X_train.shape[1] - X_test.shape[1],0)), 'constant', constant_values=(0))
X_test.shape 

(560175, 116)

In [ ]:
y_predicted = model.predict(X_test)

17506/17506 [==============================] - 737s 42ms/step


In [ ]:
# https://datascience.stackexchange.com/questions/79761/class-label-prediction-in-keras-sequential-model-showing-different-results-in-co
# https://stackoverflow.com/questions/68776790/model-predict-classes-is-deprecated-what-to-use-instead
# evidently the decided label should be the output neuron with the highest probability.
# https://datascience.stackexchange.com/questions/36238/what-does-the-output-of-model-predict-function-from-keras-mean
pred = np.argmax(y_predicted, axis=1)*2

PrepareKaggleFile(test_inputs, pred, file='gdrive/MyDrive/test_label_nn_lstm.csv')

[[     0      0]
 [     1      2]
 [     2      0]
 ...
 [560172      0]
 [560173      0]
 [560174      2]]


###Confusion matrix

In [ ]:
# Predictions
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def evaluate_lstm(model, X_test,Y_test):
  pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
  results = []
  for x in range(len(X_test)):
    result = model.predict(X_test[x].reshape(1,X_test.shape[1]),
                           batch_size=1,verbose = 3)[0]
    if np.argmax(result) == np.argmax(X_test[x]):
      if np.argmax(X_test[x]) == 0:
        neg_correct += 1
      else:
        pos_correct += 1
    if np.argmax(X_test[x]) == 0:
      neg_cnt += 1
    else:
      pos_cnt += 1
    results.append(np.argmax(result))
  Y_test_argmax = np.argmax(Y_test,axis=1)
  Y_test_argmax  = Y_test_argmax.reshape(-1,1)
  results = np.asarray(results)
  results = results.reshape(-1,1)
  conf_matrix = confusion_matrix(Y_test_argmax, results)
  fig = plt.figure(figsize=(6, 6))
  sns.heatmap(conf_matrix, annot=True, fmt="d", cmap = 'GnBu');
  plt.title("Confusion Matrix")
  plt.ylabel('Correct Class')
  plt.xlabel('Predicted class')

In [ ]:
accuracy,f1, fig = evaluate_lstm(model,X_train,Y_train)
print(f'Accuracy:{accuracy:.3f}')
print(f'F1 Score: {f1:.3f}')